# CIFAR Classification using Convolutional Nueral Networks

This notebook aims to illustrate all the different models used, their results and a discussion of the results.

First, we need to import the packages being used. `Tensorflow` for building and processing the graph, `pycf` which has the data providers and models, and a few others.

In [2]:
# Import packages
import os
import datetime
import tensorflow as tf

from pycf.utils import show_graph
from pycf.models import two_layer_fc_model
from pycf.models import four_layer_fc_model
from pycf.data_providers import CIFAR10DataProvider

# Set log level to suppress build warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

The data provider object can be easily defined as shown below. The `train_data` object is iterable over the batches. Hence, a forloop can be used for easy training.

In [3]:
# Load Training data
train_data = CIFAR10DataProvider(shape='2d')

In Tensorflow, the Graph contains all the operations. Hence, the model should be defined within the graph. Here the model `four_layer_fc_model` is used This model has four fully-connected layer. This is a very simple model which does not have enough modelling power for the task but it still can be used to confirm a correct setup.

In [4]:
# Create Graph
modelA = tf.Graph()

with modelA.as_default():
    modelInfo = four_layer_fc_model()

    # Set logging files for tensorboard
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    train_writer = tf.summary.FileWriter(os.path.join('tf-log', timestamp, 'train'), graph=modelA)
    
show_graph(modelA)

A Session is defined to train the model and the variables are initialized. Initialization is performed by running an init_op.

In [4]:
with modelA.as_default():
    sess = tf.Session()
    
    # Initialize variables
    init_op = tf.global_variables_initializer()
    sess.run(init_op)

The model can now be trained to classify the different CIFAR images. Here we are training for the `num_epoch` variable number of epochs. There are two forloops one iterating over the epochs the other over the number of batches which are provided to the `feed_dict` of the run function. This `feed_dict` supplies the data to the tensorflow _inputs_ and _targets_ placeholders defined in the model. Later we shall look a method that the data loading is part of the graph in a queue fashion.

The result still have much to be desired as the error is still very high. We shall now look at other models which can help this get lower.

In [5]:
num_epoch = 20

with modelA.as_default():
    with sess:
        for epochs in range(num_epoch):
            running_error = 0
            for batch_num, (input_batch, target_batch) in enumerate(train_data):
                _, batch_error, summary = sess.run([modelInfo.get('train_op'),
                                                    modelInfo.get('error'),
                                                    modelInfo.get('summary_op')],

                                                   feed_dict={modelInfo.get('inputs'): input_batch,
                                                              modelInfo.get('targets'): target_batch})

                running_error += batch_error
                train_writer.add_summary(summary, epochs * train_data.num_batches + batch_num)

            running_error /= train_data.num_batches
            print('End of epoch {0}: Average epoch error = {1:.2f}'.format(epochs + 1, running_error))

End of epoch 1: Average epoch error = 28.43
End of epoch 2: Average epoch error = 2.90
End of epoch 3: Average epoch error = 2.38
End of epoch 4: Average epoch error = 2.33
End of epoch 5: Average epoch error = 2.32
End of epoch 6: Average epoch error = 2.31
End of epoch 7: Average epoch error = 2.31
End of epoch 8: Average epoch error = 2.31
End of epoch 9: Average epoch error = 2.31
End of epoch 10: Average epoch error = 2.30
End of epoch 11: Average epoch error = 2.30
End of epoch 12: Average epoch error = 2.30
End of epoch 13: Average epoch error = 2.30
End of epoch 14: Average epoch error = 2.30
End of epoch 15: Average epoch error = 2.30
End of epoch 16: Average epoch error = 2.30
End of epoch 17: Average epoch error = 2.30
End of epoch 18: Average epoch error = 2.30
End of epoch 19: Average epoch error = 2.30
End of epoch 20: Average epoch error = 2.30
